# $U(1)$ Gauge Model using L2HMC in graph mode

--------------------------------------------------------------------
### TODO:
* [ ] Look at performance on Cooley (longer training runs).
* [ ] Fit observables to Eq. \ref{eq:therm_time} to determine the thermalization time $\tau$.

\begin{equation} 
f(t) \equiv A \exp^{-t / \tau}+ \,\, B
\label{eq:therm_time}
\end{equation}
* [ ] Look at defining a distance metric as the difference in topological charge between two samples and see what effect adding this as an additional term to the loss function has on the models' ability to tunnel between topological sectors.

--------------------------------------------------------------------

In [1]:
import os
import sys
import time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy.special import i0, i1

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import utils.gauge_model_helpers as helpers
from utils.data_utils import (
    calc_avg_vals_errors, block_resampling, jackknife_err
)
from gauge_model import (
    GaugeModel, check_else_make_dir, save_params_to_pkl_file
)

tfe = tf.contrib.eager
%autoreload 2
%matplotlib notebook

#from gauge_model import GaugeModel
#%reload_ext gauge_model.GaugeModel
#from gauge_model import GaugeModel
#
#from dynamics.gauge_dynamics import GaugeDynamics
#%reload_ext dynamics.gauge_dynamics
#from dynamics.gauge_dynamics import GaugeDynamics

In [2]:
def autocorr(x):
    result = np.correlate(x, x, mode='full')
    result /= result[result.argmax()]
    return result[result.size//2:]

## Define parameters

In [3]:
params = {
#--------------------- Lattice parameters ----------------------------
    'time_size': 8,
    'space_size': 8,
    'link_type': 'U1',
    'dim': 2,
    'num_samples': 4,
    'rand': False,
#--------------------- Leapfrog parameters ---------------------------
    'num_steps': 5,
    'eps': 0.2,
    'loss_scale': 1.,
    'loss_eps': 1e-4,
#--------------------- Learning rate parameters ----------------------
    'learning_rate_init': 1e-3,
    'learning_rate_decay_steps': 500,
    'learning_rate_decay_rate': 0.96,
#--------------------- Annealing rate parameters ---------------------
    'annealing': True,
    'annealing_steps': 500,
    'annealing_factor': 0.99995,
    #'beta': 3.,
    'beta_init': 3.,
    'beta_final': 8.,
#--------------------- Training parameters ---------------------------
    'train_steps': 20000,
    'save_steps': 1000,
    'logging_steps': 50,
    'training_samples_steps': 500,
    'training_samples_length': 100,
#--------------------- Model parameters ------------------------------
    'conv_net': True,
    'hmc': False,
    'eps_trainable': True,
    'metric': 'l2',
    'aux': True,
    'clip_grads': False,
    'clip_value': 10.,
}

## Create model

In [4]:
os.environ["KMP_BLOCKTIME"] = str(0)
os.environ["KMP_SETTINGS"] = str(1)
os.environ["KMP_AFFINITY"] = "granularity=fine,verbose,compact,1,0"
#os.environ["OMP_NUM_THREADS"] = str(2)

In [7]:
config = tf.ConfigProto()
tf.reset_default_graph()

In [8]:
model = GaugeModel(params=params, 
                   config=config,
                   sess=None,
                   log_dir=None,
                   restore=False)

save_params_to_pkl_file(params, model.info_dir)

Creating directory for new run: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_45/
Creating directory: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_45/samples_history.
Creating directory: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_45/samples_history/training.
Creating directory: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_45/train_samples.
time_size: 8
space_size: 8
link_type: U1
dim: 2
num_samples: 4
rand: False
num_steps: 5
eps: 0.2
loss_scale: 1.0
loss_eps: 0.0001
learning_rate_init: 0.001
learning_rate_decay_steps: 500
learning_rate_decay_rate: 0.96
annealing: True
annealing_steps: 500
annealing_factor: 0.99995
beta_init: 3.0
beta_final: 8.0
train_steps: 20000
save_steps: 1000
logging_steps: 50
training_samples_steps: 500
training_samples_length: 100
conv_net: True
hmc: False
eps_trainable: True
metric: l2
aux: True
clip_grads: False
clip_value: 10.0
--------------------------------------------------------------------------------

Building graph... (started at: Fri Feb  1 

In [ ]:
model.train(model.train_steps, kill_sess=False)

----------------------------------------------------------------------------------------------------
     STEP           LOSS       NORM. TIME     ACCEPT %        EPS           BETA           LR      
----------------------------------------------------------------------------------------------------
     0/20000       -2415         0.219        0.1376          0.2            3           0.001    
     5/20000       9753        0.0002221     3.78e-26       0.1969         3.001         0.001    
    10/20000       9878        0.0002246         0          0.1937         3.002         0.001    
    15/20000       8642        0.0002231     6.337e-27      0.1916         3.002         0.001    
    20/20000       8411        0.0002179     9.914e-29      0.1906         3.003         0.001    
    25/20000       7630        0.0002089     6.405e-26      0.1897         3.004         0.001    
    30/20000       7723        0.0002259     1.136e-19      0.1885         3.005         0.001    
    3

   400/20000       -2733       0.000244       0.9379        0.1299         3.061         0.001    
   405/20000       -2529       0.0002372      0.7332        0.1291         3.061         0.001    
   410/20000       -2580       0.0002176      0.8205        0.1283         3.062         0.001    
   415/20000       -2551       0.0002373       0.776        0.1274         3.063         0.001    
   420/20000       -2442       0.0002337      0.6695        0.1266         3.064         0.001    
   425/20000       -2518       0.0003739      0.8116        0.1259         3.064         0.001    
   430/20000       -2653       0.0002383      0.8686        0.1253         3.065         0.001    
   435/20000       -2636       0.0003109      0.9084        0.1249         3.066         0.001    
   440/20000       -2725       0.0002342      0.8833        0.1245         3.067         0.001    
   445/20000       -2753       0.0002392      0.8945         0.124         3.067         0.001    
   450/200

   690/20000       -2553       0.0002169      0.7197        0.1003         3.105        0.00096   
   695/20000       -2498       0.0002224      0.9203          0.1          3.106        0.00096   
   700/20000       -2678       0.0002148      0.8617        0.09972        3.107        0.00096   
   705/20000       -2735       0.0002242      0.8832        0.0993         3.108        0.00096   
   710/20000       -2727       0.000237       0.9131        0.09871        3.108        0.00096   
   715/20000       -2592       0.000214       0.8989        0.0981         3.109        0.00096   
   720/20000       -2645       0.0002184      0.7402        0.09754        3.11         0.00096   
   725/20000       -2843       0.0002145      0.9077        0.09698        3.111        0.00096   
   730/20000       -2755       0.0002091       0.914        0.0964         3.112        0.00096   
   735/20000       -2713        0.00023       0.8521        0.09588        3.112        0.00096   
   740/200

  1000/20000       -2483       0.0002402       0.948        0.07559        3.154       0.0009216  
  1005/20000       -2899       0.0002144      0.8667        0.07529        3.155       0.0009216  
  1010/20000       -2733       0.000235        0.978        0.07503        3.155       0.0009216  
  1015/20000       -2691       0.0002209      0.8805        0.07481        3.156       0.0009216  
  1020/20000       -2659       0.0002401      0.8611        0.0745         3.157       0.0009216  
  1025/20000       -2794       0.0002506      0.9227        0.07428        3.158       0.0009216  
  1030/20000       -2652       0.0002309         1          0.07401        3.159       0.0009216  
  1035/20000       -2702       0.0002504      0.9497        0.0738         3.159       0.0009216  
  1040/20000       -2700       0.0002379      0.9104        0.07362        3.16        0.0009216  
  1045/20000       -2801       0.0002183      0.9591        0.07337        3.161       0.0009216  
  1050/200

  1415/20000       -2648       0.0002126      0.9682        0.07072        3.22        0.0009216  
  1420/20000       -2738       0.0002159      0.8677        0.07029        3.221       0.0009216  
  1425/20000       -2732       0.0002357      0.9179        0.06966        3.222       0.0009216  
  1430/20000       -2508       0.0002132      0.9245        0.06904        3.222       0.0009216  
  1435/20000       -2753       0.0002456         1          0.06866        3.223       0.0009216  
  1440/20000       -2584       0.0002169      0.8132        0.06851        3.224       0.0009216  
  1445/20000       -2607       0.0002365      0.8879        0.06833        3.225       0.0009216  
  1450/20000       -2531        0.00021       0.8487        0.06809        3.226       0.0009216  
  1455/20000       -2750       0.0002262      0.9952        0.06776        3.226       0.0009216  
  1460/20000       -2551       0.0002258         1          0.06737        3.227       0.0009216  
  1465/200

...

 16415/20000       -2740       0.0002078         1          0.06683        6.817       0.0002708  
 16420/20000       -2684       0.0002048      0.9773        0.06684        6.818       0.0002708  
 16425/20000       -2423       0.0002923      0.9504        0.0667         6.82        0.0002708  
 16430/20000       -2687       0.0002286      0.9913        0.06652        6.822       0.0002708  
 16435/20000       -2484       0.000202       0.8304        0.06634        6.824       0.0002708  
 16440/20000       -2765       0.0002812      0.9396        0.06617        6.825       0.0002708  
 16445/20000       -2707       0.0002544      0.9618        0.06597        6.827       0.0002708  
 16450/20000       -2566       0.0003829       0.847        0.06581        6.829       0.0002708  
 16455/20000       -2783       0.000203       0.9045        0.06565        6.83        0.0002708  
 16460/20000       -2521       0.0002142      0.8486        0.0656         6.832       0.0002708  
 16465/200

 16705/20000       -2562       0.0002352      0.9661        0.06132        6.916        0.00026   
 16710/20000       -2522       0.0001995      0.9144        0.06124        6.918        0.00026   
 16715/20000       -2791       0.0002195      0.9456        0.06128        6.92         0.00026   
 16720/20000       -2780       0.0002475      0.9588        0.06125        6.922        0.00026   
 16725/20000       -2862       0.0002563      0.9226        0.06126        6.923        0.00026   
 16730/20000       -2631       0.0001998      0.9586        0.06138        6.925        0.00026   
 16735/20000       -2627       0.0002215      0.9071        0.06156        6.927        0.00026   
 16740/20000       -2359       0.0001983      0.8943        0.06162        6.928        0.00026   
 16745/20000       -2560       0.000225       0.9539        0.06167        6.93         0.00026   
 16750/20000       -2701       0.0002682       0.859        0.0617         6.932        0.00026   
 16755/200

 17000/20000       -2505       0.0001965      0.9152        0.06284        7.019       0.0002496  
 17005/20000       -2277       0.0002616      0.9338        0.06277        7.021       0.0002496  
 17010/20000       -2664       0.0002691      0.9321        0.06273        7.023       0.0002496  
 17015/20000       -2891       0.0002165         1          0.06274        7.024       0.0002496  
 17020/20000       -2566       0.0002153      0.9974        0.06283        7.026       0.0002496  
 17025/20000       -2401       0.0002413      0.9017        0.06292        7.028       0.0002496  
 17030/20000       -2634       0.0002289      0.8824        0.06306        7.03        0.0002496  
 17035/20000       -2548       0.0002005      0.9555        0.06317        7.031       0.0002496  
 17040/20000       -2615       0.0002515      0.9226        0.06325        7.033       0.0002496  
 17045/20000       -2505       0.000255       0.9533        0.06335        7.035       0.0002496  
 17050/200

 17415/20000       -2580       0.0001926      0.8878        0.07568        7.166       0.0002496  
 17420/20000       -2636       0.0001996      0.8766        0.07602        7.168       0.0002496  
 17425/20000       -2631       0.0002164      0.9077        0.0745         7.17        0.0002496  
 17430/20000       -2552       0.0001805      0.8533        0.07375        7.172       0.0002496  
 17435/20000       -2728       0.0002059      0.8164        0.07353        7.173       0.0002496  
 17440/20000       -2520       0.0002154      0.8966        0.0735         7.175       0.0002496  
 17445/20000       -2630       0.0002028      0.8294        0.07358        7.177       0.0002496  
 17450/20000       -2511       0.0002223      0.8153        0.07371        7.179       0.0002496  
 17455/20000       -2440       0.0001966      0.7163        0.07389        7.181       0.0002496  
 17460/20000       -2578       0.0002016      0.7917        0.07404        7.182       0.0002496  
 17465/200

 17710/20000       -2649       0.000342        0.932        0.07823        7.273       0.0002396  
 17715/20000       -2280       0.0003158      0.9468        0.07807        7.275       0.0002396  
 17720/20000       -2493       0.0002621      0.9671        0.0779         7.276       0.0002396  
 17725/20000       -1973       0.0002333       0.894        0.07797        7.278       0.0002396  
 17730/20000       -2520       0.000211       0.9403        0.07784        7.28        0.0002396  
 17735/20000       -2582       0.0002686      0.9655        0.07801        7.282       0.0002396  
 17740/20000       -2655       0.0002568      0.9254        0.07827        7.284       0.0002396  
 17745/20000       -2508       0.0002445         1          0.07863        7.285       0.0002396  
 17750/20000       -2178       0.0003001      0.7721        0.07906        7.287       0.0002396  
 17755/20000       -2605       0.0002729      0.9176        0.07957        7.289       0.0002396  
 17760/200

In [8]:
run_steps = [100, 200, 500, 1000, 5000]#, 10000]

In [9]:
for steps in run_steps:
    _ = model.run(steps)

Running (trained) L2HMC sampler for 100 steps...
step:      0/100    accept prob (avg):     1     step size: 0.06524 	 time/step: 0.2692

accept prob:  [1. 1. 1. 1.]


step:     10/100    accept prob (avg):  0.7109   step size: 0.06524 	 time/step: 0.08631

accept prob:  [1.         0.3958669  0.44786993 1.        ]


step:     20/100    accept prob (avg):  0.8793   step size: 0.06524 	 time/step: 0.09455

accept prob:  [0.9436177 0.7929777 1.        0.7806063]


step:     30/100    accept prob (avg):  0.9579   step size: 0.06524 	 time/step: 0.0976

accept prob:  [0.960538  0.8712547 1.        1.       ]


step:     40/100    accept prob (avg):  0.8885   step size: 0.06524 	 time/step: 0.09354

accept prob:  [0.92611206 0.82737195 0.96410793 0.83647126]


step:     50/100    accept prob (avg):  0.7495   step size: 0.06524 	 time/step: 0.08947

accept prob:  [0.5181984  0.7114266  0.76847714 1.        ]


step:     60/100    accept prob (avg):  0.8164   step size: 0.06524 	 time/step: 

step:    210/500    accept prob (avg):  0.9855   step size: 0.06524 	 time/step: 0.1033

accept prob:  [1.         1.         0.997793   0.94438213]


step:    220/500    accept prob (avg):  0.8187   step size: 0.06524 	 time/step: 0.1298

accept prob:  [0.8527701  1.         0.67327094 0.74876714]


step:    230/500    accept prob (avg):  0.8246   step size: 0.06524 	 time/step: 0.1039

accept prob:  [0.7677288 0.5307968 1.        1.       ]


step:    240/500    accept prob (avg):  0.9065   step size: 0.06524 	 time/step: 0.1103

accept prob:  [1.         1.         0.9788513  0.64696145]


step:    250/500    accept prob (avg):  0.9388   step size: 0.06524 	 time/step: 0.1071

accept prob:  [1.        0.9253192 1.        0.8297827]


step:    260/500    accept prob (avg):  0.9268   step size: 0.06524 	 time/step: 0.1259

accept prob:  [0.7073134 1.        1.        1.       ]


step:    270/500    accept prob (avg):  0.9509   step size: 0.06524 	 time/step: 0.1136

accept prob:  [1.

step:    240/1000   accept prob (avg):  0.9212   step size: 0.06524 	 time/step: 0.07646

accept prob:  [0.7390289 1.        0.9456023 1.       ]


step:    250/1000   accept prob (avg):  0.9601   step size: 0.06524 	 time/step: 0.1109

accept prob:  [1.        1.        0.8404806 1.       ]


step:    260/1000   accept prob (avg):  0.6652   step size: 0.06524 	 time/step: 0.1173

accept prob:  [0.5376011  0.6052921  1.         0.51789546]


step:    270/1000   accept prob (avg):  0.8719   step size: 0.06524 	 time/step: 0.1183

accept prob:  [0.9257691  1.         0.757105   0.80476624]


step:    280/1000   accept prob (avg):   0.761   step size: 0.06524 	 time/step: 0.1153

accept prob:  [0.6664765 1.        0.6355843 0.741767 ]


step:    290/1000   accept prob (avg):  0.9808   step size: 0.06524 	 time/step: 0.1099

accept prob:  [1.        0.9230619 1.        1.       ]


step:    300/1000   accept prob (avg):  0.9713   step size: 0.06524 	 time/step:  0.12 

accept prob:  [1.   

step:    790/1000   accept prob (avg):   0.877   step size: 0.06524 	 time/step: 0.0898

accept prob:  [1.         0.88800544 0.61985964 1.        ]


step:    800/1000   accept prob (avg):  0.6771   step size: 0.06524 	 time/step: 0.1055

accept prob:  [0.6544599  0.28860044 1.         0.7651986 ]


step:    810/1000   accept prob (avg):  0.7516   step size: 0.06524 	 time/step: 0.1222

accept prob:  [0.8249584 0.6789088 0.8297152 0.672771 ]


step:    820/1000   accept prob (avg):  0.8259   step size: 0.06524 	 time/step: 0.1024

accept prob:  [0.5674705  1.         0.83164227 0.9043746 ]


step:    830/1000   accept prob (avg):  0.9754   step size: 0.06524 	 time/step: 0.1123

accept prob:  [0.90174085 1.         1.         1.        ]


step:    840/1000   accept prob (avg):   0.99    step size: 0.06524 	 time/step: 0.1145

accept prob:  [1.         0.9949596  1.         0.96495616]


step:    850/1000   accept prob (avg):  0.9033   step size: 0.06524 	 time/step: 0.1059

accept pr

step:    320/5000   accept prob (avg):  0.8665   step size: 0.06524 	 time/step: 0.1634

accept prob:  [0.65089273 0.81528836 1.         1.        ]


step:    330/5000   accept prob (avg):   0.797   step size: 0.06524 	 time/step: 0.1435

accept prob:  [0.6351655  1.         0.5946929  0.95816636]


step:    340/5000   accept prob (avg):  0.8233   step size: 0.06524 	 time/step: 0.08927

accept prob:  [1.         0.8614407  0.47798055 0.95366347]


step:    350/5000   accept prob (avg):  0.7909   step size: 0.06524 	 time/step: 0.1021

accept prob:  [0.697138   1.         0.46644154 1.        ]


step:    360/5000   accept prob (avg):  0.9482   step size: 0.06524 	 time/step: 0.08341

accept prob:  [1.        1.        0.7929832 1.       ]


step:    370/5000   accept prob (avg):     1     step size: 0.06524 	 time/step: 0.1142

accept prob:  [1. 1. 1. 1.]


step:    380/5000   accept prob (avg):  0.8667   step size: 0.06524 	 time/step: 0.06758

accept prob:  [0.7896141 1.        0.6

...

step:   2560/5000   accept prob (avg):  0.8288   step size: 0.06524 	 time/step: 0.1427

accept prob:  [0.83764625 0.7164204  0.86352104 0.8974368 ]


step:   2570/5000   accept prob (avg):   0.717   step size: 0.06524 	 time/step: 0.1437

accept prob:  [0.76164114 0.60590124 0.5003981  1.        ]


step:   2580/5000   accept prob (avg):   0.934   step size: 0.06524 	 time/step: 0.07288

accept prob:  [1.        0.838192  0.8979247 1.       ]


step:   2590/5000   accept prob (avg):   0.793   step size: 0.06524 	 time/step: 0.1079

accept prob:  [1.        1.        0.6122721 0.5595877]


step:   2600/5000   accept prob (avg):  0.9164   step size: 0.06524 	 time/step: 0.1136

accept prob:  [0.88046646 1.         0.78499854 1.        ]


step:   2610/5000   accept prob (avg):  0.9537   step size: 0.06524 	 time/step: 0.1151

accept prob:  [1.        1.        0.8148733 1.       ]


step:   2620/5000   accept prob (avg):  0.8349   step size: 0.06524 	 time/step: 0.1538

accept prob:  [0

step:   3120/5000   accept prob (avg):  0.7579   step size: 0.06524 	 time/step: 0.105 

accept prob:  [0.8309513  0.20045346 1.         1.        ]


step:   3130/5000   accept prob (avg):   0.833   step size: 0.06524 	 time/step: 0.1159

accept prob:  [0.7176986  1.         0.76801264 0.84631306]


step:   3140/5000   accept prob (avg):  0.6933   step size: 0.06524 	 time/step: 0.1124

accept prob:  [0.5842116  0.7496756  1.         0.43919352]


step:   3150/5000   accept prob (avg):  0.8052   step size: 0.06524 	 time/step: 0.1053

accept prob:  [0.6764672  1.         0.54441756 1.        ]


step:   3160/5000   accept prob (avg):  0.9265   step size: 0.06524 	 time/step: 0.1128

accept prob:  [0.8362392 0.8695792 1.        1.       ]


step:   3170/5000   accept prob (avg):  0.9192   step size: 0.06524 	 time/step: 0.1308

accept prob:  [0.96179086 1.         0.9436988  0.7711997 ]


step:   3180/5000   accept prob (avg):  0.7696   step size: 0.06524 	 time/step: 0.1558

accept pr

step:   3680/5000   accept prob (avg):  0.9251   step size: 0.06524 	 time/step: 0.1054

accept prob:  [1.         1.         0.7096058  0.99074256]


step:   3690/5000   accept prob (avg):  0.9203   step size: 0.06524 	 time/step: 0.07821

accept prob:  [0.9588278 1.        0.7225301 1.       ]


step:   3700/5000   accept prob (avg):   0.857   step size: 0.06524 	 time/step: 0.1071

accept prob:  [0.44069973 0.98735255 1.         1.        ]


step:   3710/5000   accept prob (avg):  0.8365   step size: 0.06524 	 time/step: 0.07624

accept prob:  [0.7176167 1.        0.6282548 1.       ]


step:   3720/5000   accept prob (avg):  0.7261   step size: 0.06524 	 time/step: 0.1222

accept prob:  [0.72398937 0.97261006 0.42866245 0.7792888 ]


step:   3730/5000   accept prob (avg):  0.8677   step size: 0.06524 	 time/step: 0.1142

accept prob:  [0.6192296  1.         1.         0.85138196]


step:   3740/5000   accept prob (avg):  0.9354   step size: 0.06524 	 time/step: 0.1058

accept prob

step:   4230/5000   accept prob (avg):  0.9494   step size: 0.06524 	 time/step: 0.1352

accept prob:  [1.         1.         0.79749066 1.        ]


step:   4240/5000   accept prob (avg):  0.8222   step size: 0.06524 	 time/step: 0.1124

accept prob:  [0.8455326 0.791508  1.        0.6516607]


step:   4250/5000   accept prob (avg):  0.9347   step size: 0.06524 	 time/step: 0.1029

accept prob:  [1.         0.87278396 1.         0.8660036 ]


step:   4260/5000   accept prob (avg):  0.7219   step size: 0.06524 	 time/step: 0.127 

accept prob:  [1.         0.78482944 0.6569498  0.44573304]


step:   4270/5000   accept prob (avg):     1     step size: 0.06524 	 time/step: 0.0887

accept prob:  [1. 1. 1. 1.]


step:   4280/5000   accept prob (avg):   0.79    step size: 0.06524 	 time/step: 0.1112

accept prob:  [1.         0.86140746 0.5005564  0.79814124]


step:   4290/5000   accept prob (avg):  0.8237   step size: 0.06524 	 time/step: 0.1062

accept prob:  [0.63957626 1.         0.94

step:   4790/5000   accept prob (avg):  0.8194   step size: 0.06524 	 time/step: 0.1364

accept prob:  [1.         0.73573405 0.5417507  1.        ]


step:   4800/5000   accept prob (avg):  0.8158   step size: 0.06524 	 time/step: 0.115 

accept prob:  [0.56536055 0.70105624 1.         0.99687934]


step:   4810/5000   accept prob (avg):  0.8864   step size: 0.06524 	 time/step: 0.1067

accept prob:  [0.6371879 1.        0.908397  1.       ]


step:   4820/5000   accept prob (avg):  0.8555   step size: 0.06524 	 time/step: 0.0876

accept prob:  [1.         1.         0.42215696 1.        ]


step:   4830/5000   accept prob (avg):   0.741   step size: 0.06524 	 time/step: 0.112 

accept prob:  [0.6018602  0.8450958  0.78147334 0.7354654 ]


step:   4840/5000   accept prob (avg):  0.6436   step size: 0.06524 	 time/step: 0.07935

accept prob:  [0.60730165 0.9253264  0.60747564 0.43448862]


step:   4850/5000   accept prob (avg):  0.8815   step size: 0.06524 	 time/step: 0.1182

accept p

In [39]:
import pickle

with open(model.files['parameters_pkl_file'], 'wb') as f:
    pickle.dump(model.params, f)

In [ ]:
model.sess.graph.collections

In [ ]:
model.sess.graph.get_collection

In [ ]:
model.dynamics.position_fn.summary()

In [ ]:
# Iterate over samples history and calculate observables for each sample.
# `lattice.calc_plaq_observables(samples)` calculates observables for each of
# the samples in the mini-batch.
actions_history = []
avg_plaquettes_history = []
top_charges_history = []
for idx, samples in enumerate(samples_history):
    t0 = time.time()
    observables = np.array(model.lattice.calc_plaq_observables(samples))
    actions, plaqs, charges = observables
    
    actions_history.append(actions)
    avg_plaquettes_history.append(plaqs)
    top_charges_history.append(charges)
    
    print(f'step: {idx}  '
          f'time / step: {time.time() - t0:^6.4g}  '
          f'avg action: {np.mean(actions):^6.4g}  '
          f'avg plaquette: {np.mean(plaqs):^6.4g} '
          f'top charge: {np.mean(charges):^6.4g}')

In [ ]:
_ = helpers.plot_run_data(model.data, 
                          model.params, 
                          model.steps_arr, 
                          model.figs_dir, 
                          skip_steps=1)

In [ ]:
#tf.reset_default_graph()
#model = GaugeModel(params=params,
#                   config=None,
#                   sess=None,
#                   conv_net=False,
#                   hmc=False,
#                   log_dir='../../gauge_logs_graph/run_25',
#                   restore=True)

In [ ]:
samples = np.random.randn(*model.samples.shape)
samples_history = []

In [ ]:
for i in range(500):
    t0 = time.time()
    samples = model.sess.run(model.x_out, feed_dict={model.x: samples})
    samples_history.append(samples)
    print(f'step: {i:^6.4g} time/step: {time.time() - t0:^6.4g}')

In [ ]:
samples_history_conv = np.array(samples_history_conv)
print(samples_history_conv.shape)

In [ ]:
import pickle
samples_history_file = os.path.join(model.info_dir, 'samples_history.pkl')
with open(samples_history_file, 'wb') as f:
    pickle.dump(samples_history_conv, f)